In [1]:
%%file perceptron.py
import pickle
import numpy as np

class Perceptron():
    
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    
    def fit(self, X, y):
        self.w_ = np.random.uniform(-1.0, 1.0, 1 + X.shape[1]) 
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]
    
    def predict(self, X):
        return np.where(self.net_input(X) >= 0.0, 1, -1)

    
X_train = np.array([[0, 0], [1, 1]])
y_train = np.array([0, 1])
model = Perceptron()
model.fit(X_train, y_train)

with open('perceptron.pkl', 'wb') as f:
    pickle.dump(model, f)

Overwriting perceptron.py


In [2]:
%%file app.py

from flask import Flask, request, jsonify
import pickle
import numpy as np
from perceptron import Perceptron



app = Flask(__name__)

@app.route('/predict_get', methods=['GET'])
def get_prediction():
    sepal_length = float(request.args.get('sl'))
    petal_length = float(request.args.get('pl'))
    
    features = np.array([sepal_length, petal_length])

    with open('perceptron.pkl', 'rb') as picklefile:
        model = pickle.load(picklefile)
        
    predicted_class = int(model.predict(features))
    
    return jsonify(features=features.tolist(), predicted_class=predicted_class)

@app.route('/predict_post', methods=['POST'])
def post_predict():
    data = request.get_json(force=True)
    sepal_length = float(data.get('sl'))
    petal_length = float(data.get('pl'))
    
    features = np.array([sepal_length, petal_length])

    with open('perceptron.pkl', 'rb') as picklefile:
        model = pickle.load(picklefile)
        
    predicted_class = int(model.predict(features))
    output = dict(features=features.tolist(), predicted_class=predicted_class)
    return jsonify(output)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


Overwriting app.py


In [3]:
%%file Dockerfile
FROM python:3.11-slim-buster

WORKDIR /app

COPY requirements.txt ./
COPY app.py perceptron.pkl perceptron.py ./

RUN pip install -r requirements.txt

ENV FLASK_APP=app

EXPOSE 5000
CMD ["flask", "run", "--host", "0.0.0.0", "--port", "5000"]

Overwriting Dockerfile


In [7]:
%%file requirements.txt
Flask
numpy

Overwriting requirements.txt


In [11]:
import requests
response = requests.get("http://127.0.0.1:5000/predict_get?sl=6.3&pl=2.6")
print(response.content)
json = {"sl":2.4, "pl":2.6}
response = requests.post("http://127.0.0.1:5000/predict_post", json=json)
print(response.content)

b'{"features":[6.3,2.6],"predicted_class":-1}\n'
b'{"features":[2.4,2.6],"predicted_class":-1}\n'
